In [1]:
#ipynb config
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

from models.base import Base
from sklearn.cross_validation import train_test_split

plt.style.use('ggplot')

%matplotlib inline

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


In [3]:
if os.path.exists('../data/X.npy'):
    dataset = np.load('../data/X.npy')
    targets = np.load('../data/targets.npy')
else:
    noise = np.load('../../The-Quest-For-Mu/noise200_200.npy')
    artificial_tracks = np.load('../../The-Quest-For-Mu/convolves2.npy')
    
    def gen_big_pictures_dataset(tracks, noise, size, tracks_count, shape=(200, 200)):
        assert(len(shape) == 2)
        assert(shape[0] == shape[1])

        dataset = []
        targets = []

        past_time = time.time()
        for j in xrange(size):
            if j % 100 == 0:
                print(j, time.time() - past_time)
                past_time = time.time()

            template = np.zeros(shape)
            answer = np.zeros(shape)
            noise_template = noise[np.random.choice(np.arange(noise.shape[0]))].reshape(200, 200)
            signal_samples = artificial_tracks[np.random.choice(np.arange(artificial_tracks.shape[0]),
                                                                replace=False,
                                                                size=tracks_count)]

            if tracks_count > 0:
                tlx = np.random.choice(np.arange(shape[0] - signal_samples[0].shape[0]),
                                       replace=False,
                                       size=tracks_count)
                tly = np.random.choice(np.arange(shape[1] - signal_samples[0].shape[1]),
                                       replace=False,
                                       size=tracks_count)

            for i in xrange(tracks_count):
                answer[tlx[i]:tlx[i] + signal_samples[0].shape[0],\
                       tly[i]:tly[i] + signal_samples[0].shape[1]] = (signal_samples[i] > 0).astype(np.int8)
                template[tlx[i]:tlx[i] + signal_samples[0].shape[0],\
                         tly[i]:tly[i] + signal_samples[0].shape[1]] = signal_samples[i]


            sample = np.max([5 * template / (1e-6 + template.max() - template.min()),
                             6 * noise_template / (1e-6 + noise_template.max() - noise_template.min())],
                            axis=0)

            dataset.append(sample.ravel())
            targets.append(answer.ravel())

        return np.vstack(dataset), np.vstack(targets)
    
    
    dataset, targets = gen_big_pictures_dataset(artificial_tracks, noise, 10000, 3)